In [ ]:
import numpy as np
import matplotlib as mpl
import importlib, os, datetime
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
from quick_sim import setup_sim

from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, coupled_flux_qubit_non_linear_approx_pot_break_down

import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from PARAMETER_INPUT import *
from PARAMETER_INPUT import _lambda, _theta, _eta
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation


coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

rc_dict = {'font.size':16, 'axes.labelsize':'large', 'ytick.right':False,'legend.loc':'upper right', 'legend.fontsize':'xx-small', 'figure.autolayout':True, 'figure.figsize': (10,10), 'mathtext.fontset':'stix', 'font.family':'STIXGeneral'}
mpl.rcParams.update(rc_dict)

In [ ]:
# 4. Szilard engine test

# parameters

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
k_BT = k_B * T

C_factor = 1
L_factor = 0.35
# I_m_factor = 50
# I_m_factor = 15
I_m_factor = 0
time_scale = 1.0
R_factor = 0.05

I_p_1, I_p_2 = 2e-6 , 2e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                                # Amp
# R_1, R_2 = 1, 1  
R_1, R_2 = 371 * R_factor, 371 * R_factor
C_1, C_2 = 4e-9 * C_factor, 4e-9 * C_factor                              # F
L_1, L_2 = 1e-9 * L_factor, 1e-9 * L_factor                             # H 
gamma = 10

In [ ]:
m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;


_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4]) 
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

v_1 = np.random.normal(0, np.sqrt(k_BT/m_1)) / v_c
v_2 = np.random.normal(0, np.sqrt(k_BT/m_2)) / v_c
v_3 = np.random.normal(0, np.sqrt(k_BT/(m_1/4))) / v_c
v_4 = np.random.normal(0, np.sqrt(k_BT/(m_2/4))) / v_c

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 100
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::100] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state
params['params_key_dict'] = protocol_key_dict # the key of the params, for reference

# params['as_step'] = np.s_[::100]


# define potential

In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
phi_1_dcx , phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

phi_1_x_off, phi_2_x_off = 0, 0
phi_1_dcx_off, phi_2_dcx_off, M_12_off = 0, 0, 0

tilt_factor = 7

if I_m_factor == 0:
    phi_1_x_on = 0
    phi_2_x_on = 0
else:
    phi_1_x_on = -0.046710526315789 * tilt_factor
    phi_2_x_on = -0.046842105263157005 * tilt_factor



In [ ]:
phi_1_x_on = -0.1947
phi_2_x_on = -0.1947
# phi_2_x_on = -1
phi_1_x_on = -0
phi_2_x_on = -0



phi_1_dcx_on = 3.0
phi_2_dcx_on = 3.0
M_12_on = -0.6
    
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": phi_1_x_off,  "phi_2_x": phi_2_x_off,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}



In [ ]:
zeroDissipation = False
saveAllStates = True
params['sim_params'] = [_lambda, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "phi_1_x_on": phi_1_x_on, "phi_2_x_on": phi_1_x_on,
    "phi_1_dcx_on": phi_1_dcx_on, "phi_2_dcx_on": phi_2_dcx_on, "M_12_on": M_12_on,
    "gamma": gamma
}
    
TR_initial_condition = [
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), # t_0
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_off), # t_1
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_on),  # t_2
    (phi_1_dcx_off, phi_2_dcx_off, M_12_on),  # t_3
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_on,  phi_2_dcx_off, M_12_off)]

protocol_index = 0

def protocol_maker(time, _p):
    return {
        "duration": time, "name": _p["name"], "phi_1_x": _p["phi_1_x"], "phi_2_x": _p["phi_2_x"],
        "M_12": _p["M_12"], "phi_1_dcx": _p["phi_1_dcx"], "phi_2_dcx": _p["phi_2_dcx"]
    }


_P = [
    {"duration":50 * time_scale, "name":"return to 4 well potential (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_off , "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off,  
    },
    
    # forward
    {"duration": 50 * time_scale, "name":"(t_1) mix in y direction (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_on * 0.1, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  
    }, # 1
    {"duration":200 * time_scale, "name":"(t_2) mix in y direction (b)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_on, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  
    }, # 2
    {"duration":50 * time_scale, "name":"(t_3) conditional tilt",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_on, "M_12": M_12_on,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  
    }, # 3
    {"duration":50 * time_scale, "name":"(t_4) return to 4 well",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_off, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off,  
    }, # 4
    
    {"duration":50 * time_scale, "name":"(t_5) conditional tilt",
        "phi_1_x": phi_1_x_on, "phi_2_x": phi_2_x_off, "M_12": M_12_on,  
        "phi_1_dcx": phi_1_dcx_on, "phi_2_dcx": phi_2_dcx_off,  
    }, # 5
    {"duration":50 * time_scale, "name":"(t_6) mix in x direction",
        "phi_1_x": phi_1_x_on, "phi_2_x": phi_2_x_off, "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_on, "phi_2_dcx": phi_2_dcx_off,  
    }, 
    {"duration":200 * time_scale, "name":"return to 4 well potential (a)",
        "phi_1_x": phi_1_x_on * 0.1, "phi_2_x": phi_2_x_off , "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off,  
    },
    {"duration":50 * time_scale, "name":"return to 4 well potential (a)",
        "phi_1_x": phi_1_x_off, "phi_2_x": phi_2_x_off , "M_12": M_12_off,  
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off,  
    }
]

# bookmark


initial_condition = _P[0]
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
initial_condition["phi_1_dcx"], initial_condition["phi_2_dcx"], initial_condition["M_12"]

protocol_list = [
    # protocol_maker(50, _P[1]), protocol_maker(200, _P[2]), 
    # protocol_maker(50, _P[3]),
    protocol_maker(50, _P[4]), 
    # protocol_maker(50, _P[3]), 
    # protocol_maker(50, _P[2]), 
    # protocol_maker(200, _P[1]), protocol_maker(50,  _P[0])
]

# protocol_list = [
    
#     # {"duration":50 * time_scale, "phi_2_x": 0, "phi_2_dcx": 0,  "M_12": -0.6,  "name":"(1) mix in y direction (a)"},    
#     {"duration":50 * time_scale, "phi_2_x": phi_1_x_on * 0.1,  "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name":"(1) mix in y direction (a)"},
#     {"duration":200 * time_scale,"phi_2_x": phi_1_x_on, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction (b)"},
#     {"duration":50 * time_scale, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt"},
#     {"duration":50 * time_scale, "phi_2_x": phi_2_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(3) return to 4 well"},
#     {"duration":50 * time_scale, "phi_1_x": phi_1_x_on, "phi_2_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,  "name": "(4) conditional tilt in x"}, 
#     {"duration":50 * time_scale, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5) mix in x direction"}, 
#     {"duration":200 * time_scale, "phi_1_x": phi_1_x_on * 0.1, "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) return to 4 well potential (a)"}, 
#     {"duration":50 * time_scale,  "phi_1_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential (b)"}, 
# ]




In [ ]:
params['N'] = 1000
params['dt'] = 1/100
protocol_list = [
    
    # {"duration":50 * time_scale, "phi_2_x": 0, "phi_2_dcx": 0,  "M_12": -0.6,  "name":"(1) mix in y direction (a)"},    
    {"duration":50 * time_scale, "phi_2_x": phi_1_x_on * 0.1,  "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5, "M_12": M_12_off,  "name":"(1) mix in y direction (a)"},
    {"duration":200 * time_scale,"phi_2_x": phi_1_x_on, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction (b)"},
    {"duration":200 * time_scale, "phi_1_x": phi_1_x_on * 0.1, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5, "M_12": M_12_off,  "name":"(6) return to 4 well potential (a)"}, 
    {"duration":50 * time_scale,  "phi_1_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential (b)"}, 
]

In [ ]:
regenerate_init_state = False

In [ ]:
%%capture

# regenerate_init_state = True
if regenerate_init_state:
    initial_potential_protocol_list = [create_CE_Protocol(10, four_well)]
    init_state_saved = cfq_batch_sweep.create_initial_state(initial_parameter_dict, initial_potential_protocol_list, coupled_fq_pot, params)
else:
    init_state_saved = np.load("four_well_default_init_state_N_1000.npy")
    pass

init_state_used = init_state_saved


In [ ]:
protocol_list

In [ ]:
"""
# step 3: create the relevant storage protocol and computation protocol
"""
def modifiedFunction(t, result):
    return result


computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity)

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(init_state=init_state_saved)
cfqr.set_sim_attributes(init_state=init_state_used)


In [ ]:
cfqr.showPamameterTimeGraph(protocol_time_array, protocol_all_time_array)
# showPamameterTimeGraph(cfqr, protocol_time_array, protocol_all_time_array)

# actual simulations

In [ ]:
# step 5: perform simulations
manual_domain=[np.array([-5, -5]), np.array([5, 5])]

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param,            
                        initial_state = init_state_used, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, measure_all_states=saveAllStates, \
                        has_velocity = has_velocity)

### Work Analysis

In [ ]:
import sys, importlib
# importlib.reload(sys.modules['edward_tools.jupyter_helper_function'])
from edward_tools.jupyter_helper_function import work_analysis, fidelity_analysis

In [ ]:
fidelity_analysis(simResult, params, initial_parameter_dict, protocol_list)

# KE Analysis

In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.jupyter_helper_function'])
from edward_tools.jupyter_helper_function import work_analysis, KE_analysis, single_particle_analysis

In [ ]:
KE_analysis(simResult)

# single particle analysis

In [ ]:
single_particle_analysis(simResult, params, protocol_list)

In [ ]:
_ = simResult['cfqr'].plotGraphWithContourGraph(showGraph = True)

# state_plot_potential_KE_animation_anlaysis

In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.jupyter_helper_function'])
from edward_tools.jupyter_helper_function import work_analysis, KE_analysis, single_particle_analysis, state_plot_potential_KE_animation_anlaysis

In [ ]:
_t = 200

In [ ]:

params_at_t =  simResult['cfqr'].protocol.get_params(_t)
_phi_1_dcx = params_at_t[10]
_phi_2_dcx = params_at_t[11]
slice_value = [0, 0, _phi_1_dcx, _phi_2_dcx]
particle_k = simResult['cfqr'].get_all_state()[:, _t, ...]
cutlineValue = -2
state_plot_potential_KE_animation_anlaysis(simResult, params, _t, cutlineValue, index_label = ['00', '01', '10', '11'])

In [ ]:
pColor = {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
names = [r"$\varphi_1$", r"$\varphi_2$", r"$\varphi_{1dc}$", r"$\varphi_{2dc}$"]
fig, ax = plt.subplots(1, 2, figsize=[10, 5])
ax_flatten = ax.flatten()
all_state = simResult['cfqr'].sim.output.all_state['states']
project_item = ["00", "01"]

# title = ax_flatten[0].set_title(ax0_title)
# _line = ax_flatten[0].plot([], [], color = "red")[0]
cutlineInformation = [(-2, "v","red")]
cutlineXLimit = [-4, 4], 
cutlineYLimit = [50, 400]
offset_potential = False

N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
state_lookup = separate_by_state(all_state[:, :, (0, 1), :])
opacity = 0.5

frame_skip  = 10
index_skip_in_all_time_array = int(frame_skip /params['dt'])
time_array = protocol_all_time_array[::index_skip_in_all_time_array]

plot_axis = [0, 1]

phi_1 = all_state[:,::frame_skip, 0, 0]
phi_2 = all_state[:,::frame_skip, 1, 0]
phi_1dc = all_state[:,::frame_skip, 2, 0]
phi_2dc = all_state[:,::frame_skip, 3, 0]
parms_at_init = simResult['cfqr'].protocol.get_params(0)
particle_potential = simResult['cfqr'].system.potential.potential(phi_1, phi_2, phi_1dc, phi_2dc, parms_at_init)



phi_1_lim = (np.min(phi_1), np.max(phi_1))
phi_2_lim = (np.min(phi_2), np.max(phi_2))
scat_kwargs = {'alpha':opacity, 'zorder':10}

scat = [ax_flatten[0].scatter(phi_1[state_lookup[key], 0], phi_2[state_lookup[key], 0], c = pColor[key], **scat_kwargs) for key in state_lookup]

plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = _value,\
                        contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                        showGraph=False, cutlineColor = _color, x_range = [-4, 4], cutlineYLimit = cutlineYLimit, offset_potential = offset_potential)

scat_2 = [ax_flatten[1].scatter(phi_1[state_lookup[key], 0], particle_potential[state_lookup[key], 0], c = pColor[key], **scat_kwargs) for key in state_lookup]

_value, _direction, _color = cutlineInformation[0]
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = time_array[0], cutlineDirection = _direction)



ax_flatten[0].set(xlim=x_lim, ylim=y_lim, xlabel=names[plot_axis[0]], ylabel=names[plot_axis[1]])
ax_flatten[0].set_aspect(1)


def animate(i):
    x_i = phi_1[:, i]
    y_i = phi_2[:, i]
    
    parms_at_i = simResult['cfqr'].protocol.get_params(time_array[i])
    particle_potential_i = simResult['cfqr'].system.potential.potential(phi_1[:, i], phi_2[:, i], phi_1dc[:, i], phi_2dc[:, i], parms_at_i)

    ax_flatten[1].clear()
    scat_2 = [ax_flatten[1].scatter(phi_1[state_lookup[key], 0], particle_potential[state_lookup[key], 0], c = pColor[key], **scat_kwargs) for key in state_lookup if key in project_item]
    for item in cutlineInformation:
        _value, _direction, _color = item
        
        X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = time_array[i], cutlineDirection = _direction)

        plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = _value,\
                                contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                                showGraph=True, cutlineColor = _color, x_range = [-4, 4], cutlineYLimit = cutlineYLimit, offset_potential = offset_potential)

    params_at_t_i = simResult['cfqr'].protocol.get_params(time_array[i])
    # potential_at_t_i = simResult['cfqr'].system.potential.potential(x_1, y_i)
        
    for _i, item in enumerate(state_lookup):
        scat[_i].set_offsets(np.c_[x_i[state_lookup[item]], y_i[state_lookup[item]]])
        
        if item in project_item:
            scat_2[_i].set_offsets(np.c_[y_i[state_lookup[item]], particle_potential_i[state_lookup[item]]])



ani = FuncAnimation(fig, animate, interval=100, frames=len(time_array), blit=False)

In [ ]:
particle_potential[state_lookup["01"], 0].shape

In [ ]:
particle_potential[state_lookup["01"], 0].shape

In [ ]:
phi_1[state_lookup["01"], 0].shape

In [ ]:
display.HTML(ani.to_html5_video())

In [ ]:
def animate_sim_flux_qubit_with_cutline(cfqr, frame_skip=30, time_array=None, params = None, legend=True, plot_axis = [0, 1],
                                        slice_values = None, fig_ax=None, save_path = None, save_dict = None, ax0_title = None,
                                        vmin = None, vmax = None, manual_domain = [np.array([-5, -5]), np.array([5, 5])], cutlineInformation = [(-3, "v","red")],
                                        cutlineGeneratingFunction = None, cutlineXLimit = [-4, 4], cutlineYLimit = None, pColor = None, 
                                        opacity = None, offset_potential = False, **pot_kwargs):

    # dynamicCutline: need to convert the minimum point format [(x_1, y_1), (x_2, y_2), ...] to [(x_1, v, red), ...]
    
    if not pColor:
        pColor = {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
    
    names = [r"$\varphi_1$", r"$\varphi_2$", r"$\varphi_{1dc}$", r"$\varphi_{2dc}$"]
    fig, ax = plt.subplots(1, 2, figsize=[10, 5])
    ax_flatten = ax.flatten()
    
    all_state = cfqr.sim.output.all_state['states']
    phi_1 = all_state[:,::frame_skip, 0, 0]
    phi_2 = all_state[:,::frame_skip, 1, 0]
    phi_1dc = all_state[:,::frame_skip, 2, 0]
    phi_2dc = all_state[:,::frame_skip, 3, 0]
    
    N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
    
    state_lookup = separate_by_state(all_state[:, :, (0, 1), :])

    if opacity == None:
        opacity = min(1, 300/N)
    
    time_array = time_array[::frame_skip]
    if cutlineGeneratingFunction:
        dynamicCutlineData = [cutlineGeneratingFunction(_t) for _t in time_array]
        
   
    phi_1_lim = (np.min(phi_1), np.max(phi_1))
    phi_2_lim = (np.min(2), np.max(phi_2))
    scat_kwargs = {'alpha':opacity, 'zorder':10}

    scat = [ax_flatten[0].scatter(phi_1[state_lookup[key], 0], phi_2[state_lookup[key], 0], c = pColor[key], **scat_kwargs) for key in state_lookup]
    
    if manual_domain:
        phi_1_lim = (manual_domain[0][0], manual_domain[1][0])
        phi_2_lim = (manual_domain[0][1], manual_domain[1][1])
        ax_flatten[0].set(xlim=phi_1_lim, ylim=phi_2_lim, xlabel=names[0], ylabel=names[1])

    # ax_flatten[0].set(xlim=phi_1_lim, ylim=phi_2_lim, xlabel=names[plot_axis[0]], ylabel=names[plot_axis[1]])
    ax_flatten[0].set_aspect(1)
    title = ax_flatten[0].set_title(ax0_title)
    _line = ax_flatten[0].plot([], [], color = "red")[0]


    
    def animate(i):
        phi_1_i = phi_1[:, i]
        phi_2_i = phi_2[:, i]
        ax_flatten[1].clear()

        
        if cutlineGeneratingFunction:
            
            _value, _direction, _color = dynamicCutlineData[i]
            
            X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, 
                                    t = time_array[i], cutlineDirection = _direction)
            
        
            plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = _value,\
                                    contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                                    showGraph=True, cutlineColor = _color, contour_range = [vmin, vmax], x_range = cutlineXLimit, 
                                    cutlineYLimit = cutlineYLimit, dynamicCutline = True, 
                                    offset_potential = offset_potential)
            plt_line_data_at_t['_line'][0].set_data([], [])
            
            if cutlineDirection == "v":
                _line.set_data([_value, _value], [-5, 5])
            if cutlineDirection == "h":
                _line.set_data([-5, 5], [_value, _value])


             # x_range = None, y_range = None
                
        else:

            for item in cutlineInformation:
                _value, _direction, _color = item
                X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = time_array[i], cutlineDirection = _direction)
            # U = U - np.min(U)

                plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = _value,\
                                        contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                                        showGraph=True, cutlineColor = _color, x_range = [-4, 4], cutlineYLimit = cutlineYLimit, offset_potential = offset_potential)

        
        
        ax_flatten[1].set_title(f"t = {time_array[i]}")
        ax_flatten[1].set_xlabel(r"$\varphi_2$")
        ax_flatten[1].set_ylabel(r"potential ($k_BT$)")
        for i, item in enumerate(state_lookup):
            scat[i].set_offsets(np.c_[phi_1_i[state_lookup[item]], phi_2_i[state_lookup[item]]])
            # ax_flatten[0].scatter(x_i[state_lookup[item]], y_i[state_lookup[item]])
            

    ani = FuncAnimation(fig, animate, interval=100, frames=len(time_array), blit=False)

    return ani, fig, ax


In [ ]:
state_potential = simResult['cfqr'].system.get_potential(all_states[:, _index_of_t, ...], _t)





pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
for _label in index_label:
    states_x_data = all_states[index_dict[_label], _index_of_t, 0,0]
    states_y_data = all_states[index_dict[_label], _index_of_t, 1,0]
    ax[0].scatter(states_x_data, states_y_data, color = pColor[_label])
    ax[1].scatter(states_y_data, state_potential[index_dict[_label]], color = pColor[_label])

In [ ]:
np.min(np.where(protocol_all_time_array[params['as_step']]==300)) 

In [ ]:
simResult['cfqr'].protocol_all_time_array

In [ ]:
def plotGraphWithContourGraph(self, t = 0, cutlineDirection = "v", cutlineValue = 0, contour_range = [0, 450], showGraph = None, saveGraph = None, graph_title = " ", x_range = None, y_range = None, mark_critical_points = None):
    fig, ax = plt.subplots(1, 2, figsize = [8, 3])


    X, Y, U, cutlineDirection, _targetAxis, _plotAxis = coupled_fq_protocol_library.get_potential_along_a_1D_cutline(self, t = t, cutlineDirection =cutlineDirection, cutlineValue = cutlineValue, resolution = 100, contours = 5, manual_domain = [[-5, -5], [5, 5]])

    plotLineData = coupled_fq_protocol_library.plotCutlines(X, Y, U, cutlineDirection = cutlineDirection, cutlineValue = cutlineValue, x_range = x_range, y_range = y_range, contour_range = contour_range, cutlineColor = 'red' , contour_plt = ax[0], cutline_plt = ax[1], contours = 5, time = None, graph_title = graph_title, showGraph = showGraph, saveGraph = saveGraph)


    if mark_critical_points:
        pass


    return plotLineData

# animation

In [ ]:
showAnimation = False
showAnimation = True

pColor = {"00": "#061DF7", "01": "silver", "10": "#04B3BC", "11": "#FCEF51"}
if showAnimation:
    simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict, pColor = pColor, frame_skip = 10)

In [ ]:
import pickle 
save_all_states = True
sim_id = simResult['simulation_data']['simulation_id']

pickle_file_name = f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_all_states_step_and_trial_data.pkl'
npy_file_name = f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_all_states.npy'
if save_all_states:
    
    all_state_step_and_trial_data = {x: simResult['cfqr'].sim.output.all_state[x] for x in ['step_indices', 'trial_indices']}
    with open(pickle_file_name, 'wb') as f:
        pickle.dump(all_state_step_and_trial_data, f)
    np.save(npy_file_name, simResult['cfqr'].sim.output.all_state['states'])

In [ ]:
with open(f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id}_all_states_step_and_trial_data.pkl', 'rb') as f:
    loaded_all_state_step_and_trial_data = pickle.load(f)


In [ ]:
os.system(f"ls -alh {npy_file_name}")

In [ ]:
os.system(f"rm {npy_file_name}")

In [ ]:
loaded_all_state_step_and_trial_data

In [ ]:
simResult['cfqr'].get_all_state().shape

In [ ]:
# simResult['cfqr'].getFidelityBarChart()

In [ ]:
# simResult['cfqr'].getFidelityTimeGraph()

In [ ]:
# pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
# simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict, pColor = pColor, frame_skip= 10)

In [ ]:
# # cutline graph
# # , (1.96, "v","orange")
# pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
# importlib.reload(coupled_fq_protocol_library)
# ani,_,_ =  coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline(simResult['cfqr'], frame_skip = 5, time_array=simResult['cfqr'].sim.target_step_index, 
#                                                                            params = params, cutlineInformation = [(-1.96, "v","red") , (1.96, "v","orange")],
#                                                                           pColor=pColor, opacity = 0, cutlineYLimit = [0, 250])
# video = ani.to_html5_video()
# html = display.HTML(video)
# display.display(html)

In [ ]:
# from edward_tools.visualization import separate_by_state
# # fig, ax = plt.subplots(1, 2, figsize =  [8, 4])
# frame_skip = 10
# plot_distribution_with_cutline(simResult['cfqr'], 0, time_array=protocol_all_time_array, pcolor=pColor, cutlineInformation = [-1.964, "v", "red"])

In [ ]:
# def plot_distribution_with_cutline(self, t, time_array=None, params = None, legend=False, slice_values = None, fig_ax=None, 
#                                    vmin = None, vmax = None, 
#                                    cmap = "afmhot", 
#                                    pcolor = {"00": "blue", "01": "orange", "10": "green", "11": "red"}, 
#                                    cutlineInformation = [-3, "v","red"], **pot_kwargs):
#     names = [r"$\varphi_1$", r"$\varphi_2$"]
#     fig, ax = plt.subplots(1, 2, figsize=[8, 3.5])
#     ax_flatten = ax.flatten()
#     manual_domain=[np.array([-5, -5]), np.array([5, 5])]

#     all_state = self.sim.output.all_state['states']
#     phi_1_dc = all_state[..., 2, 0]
#     phi_2_dc = all_state[..., 3, 0]
#     all_state = all_state[:, :, (0, 1), :]

#     N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
#     which_axes = [np.s_[..., i, 0] for i in range(N_dim)]
#     x_array = [all_state[item] for item in which_axes]
#     state_lookup = separate_by_state(all_state)
#     opacity = min(1, 300/N)

#     time_array = time_array[::frame_skip]
#     x = x_array[0][:,::frame_skip]
#     y = x_array[1][:,::frame_skip]
#     x_lim = (np.min(x), np.max(x))
#     y_lim = (np.min(y), np.max(y))
#     scat_kwargs = {'alpha':opacity, 'zorder':10}
#     scat = [ax_flatten[0].scatter(x[state_lookup[key], 0], y[state_lookup[key], 0], c = pColor[key], **scat_kwargs) for key in state_lookup]
#     # scat_1 = [ax_flatten[1].scatter(x[state_lookup[key], 0], y[state_lookup[key], 0], **scat_kwargs) for key in state_lookup]

#     modified_manual_domain = [(manual_domain[0][1], manual_domain[0][0]), \
#                               (manual_domain[1][1], manual_domain[1][0])]

# #     resolution = 100
# #     U, X_mesh = self.system.lattice(t, resolution, axes=(0, 1),\
# #                                 manual_domain=modified_manual_domain, slice_values = slice_values)
# #     X, Y = X_mesh[0], X_mesh[1]

# #     # 5. find vmin, vmax, x_min, x_max, y_min and y_max
# #     vmin, vmax = np.min(U), np.max(U)
# #     x_min, x_max = np.min(X), np.max(X)
# #     y_min, y_max = np.min(Y), np.max(Y)
# #     cont = ax_flatten[0].contourf(X, Y, U, 40, vmin = 0, vmax = 400, cmap = "afmhot")


#     X, Y, U, cutlineDirection, _targetAxis, _plotAxis = coupled_fq_protocol_library.get_potential_along_a_1D_cutline(self, t = t, cutlineDirection =cutlineInformation[1], cutlineValue = cutlineInformation[0], resolution = 100, contours = 5, manual_domain = [[-5, -5], [5, 5]])
            
#     coupled_fq_protocol_library.plotCutlines(X, Y, U, cutlineDirection = cutlineDirection, cutlineValue = cutlineInformation[0], cutlineColor = cutlineInformation[2] , contour_plt = ax[0], cutline_plt = ax[1], contours = 5, time = None, graph_title = None, showGraph = True)

    
    

### generate state at different time

In [ ]:
from edward_tools.visualization import separate_by_state
minimum_point_data = [cfqr.system.get_minimum_point(_t, coupled_flux_qubit_non_linear_approx_pot, guess=[(-2, -2)]) for _t in protocol_time_array]
all_state = simResult['cfqr'].sim.output.all_state['states'][10:50, ...]
all_state = all_state[:, :, (0, 1), :]
N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
which_axes = [np.s_[..., i, 0] for i in range(N_dim)]
x_array = [all_state[item] for item in which_axes]
state_lookup = separate_by_state(all_state)
opacity = min(1, 300/N)
x, y = x_array[0], x_array[1]
x_lim = (np.min(x), np.max(x))
y_lim = (np.min(y), np.max(y))
scat_kwargs = {'alpha':opacity, 'zorder':10}

In [ ]:

min_U_along_cutline = []
fig, ax = plt.subplots(5, 4, figsize=[14, 17])
ax_flatten = ax.flatten()
cutlineRange = [-3.5, 3.5]
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
cutline_xrange = [-3.5, 3.5]
cutline_yrange = [0, 200]

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[0], cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[0][0][0],\
                        contour_plt=ax_flatten[2*0], cutline_plt=ax_flatten[2*0 + 1], contours = 20,
                        showGraph=True, time=protocol_time_array[0], offset_potential = True, x_range = cutline_xrange, y_range = cutline_yrange, graph_title = " ")
min_U = np.min(U)
scat = [ax_flatten[2 * 0].scatter(x[state_lookup[key], protocol_time_index_array[0]], y[state_lookup[key], protocol_time_index_array[0]], c = pColor[key],**scat_kwargs) for key in state_lookup]
ax_flatten[2 * 0].set_xticks([-4, 0, 4])
ax_flatten[2 * 0].set_yticks([-4, 0, 4])
ax_flatten[2 * 0 + 1].set_yticks([0, 100, 200])
ax_flatten[2 * 0].xaxis.set_tick_params(labelsize=20)
ax_flatten[2 * 0].yaxis.set_tick_params(labelsize=20)
ax_flatten[2 * 0 + 1].set_xticks([-4, -2, 0, 2, 4])
ax_flatten[2 * 0 + 1].set_yticks([0, 100, 200])
ax_flatten[2 * 0 + 1].xaxis.set_tick_params(labelsize=20)
ax_flatten[2 * 0 + 1].yaxis.set_tick_params(labelsize=20)

for _i in range(1, 5):
    X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[_i], cutlineDirection = "v")
    
    plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[_i][0][0],\
                            contour_plt=ax_flatten[2*_i], cutline_plt=ax_flatten[2*_i + 1], contours = 20,
                            showGraph=True, time=protocol_time_array[_i], offset_potential = True, x_range = cutline_xrange, y_range = cutline_yrange, graph_title = " ")
    
    _index = list(simResult['cfqr'].sim.target_step_index).index(protocol_time_index_array[_i]+1)
    scat = [ax_flatten[2 * _i].scatter(x[state_lookup[key], _index], y[state_lookup[key], _index], c = pColor[key], **scat_kwargs) for key in state_lookup]
    ax_flatten[2 * _i].set_xticks([-4, 0, 4])
    ax_flatten[2 * _i].set_yticks([-4, 0, 4])
    ax_flatten[2 * _i + 1].set_xticks([-4, -2, 0, 2, 4])
    ax_flatten[2 * _i + 1].set_yticks([0, 100, 200])
    ax_flatten[2 * _i].tick_params(labelbottom=False, labelleft = False)    
    ax_flatten[2 * _i+1].tick_params(labelbottom=False, labelleft = False)    
    # ax_flatten[2 * _i].xaxis.set_tick_params(labelsize=16)
    # ax_flatten[2 * _i].yaxis.set_tick_params(labelsize=16)

    # ax_flatten[2 * _i + 1].set_xticks([])
    # ax_flatten[2 * _i + 1].xaxis.set_tick_params(labelsize=16)
    # ax_flatten[2 * _i + 1].yaxis.set_tick_params(labelsize=16)
    
    
for _i in range(5, 9):
    X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[_i], cutlineDirection = "h")
    plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[_i][0][1],\
                            contour_plt=ax_flatten[2*_i], cutline_plt=ax_flatten[2*_i + 1], contours = 20,
                            showGraph=True, time=protocol_time_array[_i], offset_potential = True, x_range = cutline_xrange, y_range = cutline_yrange, graph_title = " ")
    min_U = np.min(U)
    _index = list(simResult['cfqr'].sim.target_step_index).index(protocol_time_index_array[_i]+1)
    scat = [ax_flatten[2 * _i].scatter(x[state_lookup[key], _index], y[state_lookup[key], _index], c = pColor[key], **scat_kwargs) for key in state_lookup]
    ax_flatten[2 * _i].set_xticks([-4, 0, 4])
    ax_flatten[2 * _i].set_yticks([-4, 0, 4])
    ax_flatten[2 * _i + 1].set_xticks([-4, -2, 0, 2, 4])
    ax_flatten[2 * _i + 1].set_yticks([0, 100, 200])
    ax_flatten[2 * _i].tick_params(labelbottom=False, labelleft = False)    
    ax_flatten[2 * _i+1].tick_params(labelbottom=False, labelleft = False)    

    # ax_flatten[2 * _i + 2].plot(plt_line_data_at_t['cutline_plot']['plotAxis'], plt_line_data_at_t['cutline_plot']['targetU'])

In [ ]:

min_U_along_cutline = []
fig, ax = plt.subplots(1, 2, figsize=[5.7, 3])
ax_flatten = ax.flatten()
cutlineRange = [-3.5, 3.5]
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
cutline_xrange = [-3.5, 3.5]
cutline_yrange = [0, 200]

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[0], cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[0][0][0],\
                        contour_plt=ax_flatten[2*0], cutline_plt=ax_flatten[2*0 + 1], contours = 20,
                        showGraph=True, time=protocol_time_array[0], offset_potential = True, x_range = cutline_xrange, y_range = cutline_yrange, graph_title = " ")
min_U = np.min(U)
scat = [ax_flatten[0].scatter(x[state_lookup[key], protocol_time_index_array[0]], y[state_lookup[key], protocol_time_index_array[0]], c = pColor[key],**scat_kwargs) for key in state_lookup]
ax_flatten[0].set_xticks([-4, 0, 4])
ax_flatten[0].set_yticks([-4, 0, 4])
ax_flatten[1].set_yticks([0, 100, 200])
ax_flatten[0].xaxis.set_tick_params(labelsize=18)
ax_flatten[0].yaxis.set_tick_params(labelsize=18)
ax_flatten[1].set_xticks([-4, -2, 0, 2, 4])
ax_flatten[1].set_yticks([0, 100, 200])
ax_flatten[1].xaxis.set_tick_params(labelsize=18)
ax_flatten[1].yaxis.set_tick_params(labelsize=18)

plt.show()

for _i in range(1, 5):
    fig, ax = plt.subplots(1, 2, figsize=[5.7, 3])
    ax_flatten = ax.flatten()
    X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[_i], cutlineDirection = "v")
    
    plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[_i][0][0],\
                            contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                            showGraph=True, time=protocol_time_array[_i], offset_potential = True, x_range = cutline_xrange, y_range = cutline_yrange, graph_title = " ")
    
    _index = list(simResult['cfqr'].sim.target_step_index).index(protocol_time_index_array[_i]+1)
    scat = [ax_flatten[0].scatter(x[state_lookup[key], _index], y[state_lookup[key], _index], c = pColor[key], **scat_kwargs) for key in state_lookup]
    ax_flatten[0].set_xticks([-4, 0, 4])
    ax_flatten[1].set_yticks([-4, 0, 4])
    ax_flatten[1].set_xticks([-4, -2, 0, 2, 4])
    ax_flatten[1].set_yticks([0, 100, 200])
    ax_flatten[0].tick_params(labelbottom=False, labelleft = False)    
    ax_flatten[1].tick_params(labelbottom=False, labelleft = False)    
    plt.show()
    # ax_flatten[2 * _i].xaxis.set_tick_params(labelsize=16)
    # ax_flatten[2 * _i].yaxis.set_tick_params(labelsize=16)

    # ax_flatten[2 * _i + 1].set_xticks([])
    # ax_flatten[2 * _i + 1].xaxis.set_tick_params(labelsize=16)
#     # ax_flatten[2 * _i + 1].yaxis.set_tick_params(labelsize=16)
    
    
# for _i in range(5, 9):
#     X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[_i], cutlineDirection = "h")
#     plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[_i][0][1],\
#                             contour_plt=ax_flatten[2*_i], cutline_plt=ax_flatten[2*_i + 1], contours = 20,
#                             showGraph=True, time=protocol_time_array[_i], offset_potential = True, x_range = cutline_xrange, y_range = cutline_yrange, graph_title = " ")
#     min_U = np.min(U)
#     _index = list(simResult['cfqr'].sim.target_step_index).index(protocol_time_index_array[_i]+1)
#     scat = [ax_flatten[2 * _i].scatter(x[state_lookup[key], _index], y[state_lookup[key], _index], c = pColor[key], **scat_kwargs) for key in state_lookup]
#     ax_flatten[2 * _i].set_xticks([-4, 0, 4])
#     ax_flatten[2 * _i].set_yticks([-4, 0, 4])
#     ax_flatten[2 * _i + 1].set_xticks([-4, -2, 0, 2, 4])
#     ax_flatten[2 * _i + 1].set_yticks([0, 100, 200])
#     ax_flatten[2 * _i].tick_params(labelbottom=False, labelleft = False)    
#     ax_flatten[2 * _i+1].tick_params(labelbottom=False, labelleft = False)    

#     # ax_flatten[2 * _i + 2].plot(plt_line_data_at_t['cutline_plot']['plotAxis'], plt_line_data_at_t['cutline_plot']['targetU'])

In [ ]:
v_1

In [ ]:
np.mean(all_state[:, 0, 1, 1])

In [ ]:
np.mean(simResult['cfqr'].sim.output.all_state["states"][:, 0, 0, 1])

# Fidelity

# work analysis of subprocesses

In [ ]:
# step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
step_array = simResult['cfqr'].sim.target_step_index
skip_step = int(len(step_array) * 0.05)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]

plt.errorbar(step_array[::skip_step], work_mean[::skip_step], yerr = work_std[::skip_step])
# plt.yscale('log')

for i, t in enumerate(step_time_array):
    plt.axvline(x = t, color = 'b', linestyle ='--', label = 'axvline - full height')
    # plt.annotate(f"{i+1}", xy=(t, 4.5),  color='red', ha='center', size=16)

print(f'jarzyn = {jarzyn_term}')
print([x["duration"] for x in protocol_list])

In [ ]:

target_time_index = np.array(protocol_time_array)/params['dt']
target_time_index = target_time_index.astype("int") - 1

# work_done_array = [work_mean[i] for i in target_time_index[1:]]
work_done_array = simResult["cfqr"].sim.keyStep_work_statistic[:,0]
work_done_of_each_step = np.array(work_done_array)[1:] - np.array(work_done_array)[:-1]

Measure_Erasue_steps = [0, 1, 2, 3]
Control_Steps = [4, 5, 6, 7]

W_M = work_done_array[4] - work_done_array[0]
W_C = work_done_array[-1] - work_done_array[4]

print("Third way to split the steps: (E, 0-1), (M. 2), (C: 3-6)")
print(f"work_done = {work_done_of_each_step}" )
print(f"measurement + reset = {W_M}, deviation: {W_M - 0.693}")
print(f"control = {W_C}, deviation: {0.693 + W_C}")
print(sum(work_done_of_each_step))

# animations

In [ ]:
simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict)

In [ ]:
from edward_tools.visualization import separate_by_state

In [ ]:
self = simResult['cfqr']
t = 100
resolution = 100

fig, ax = plt.subplots(figsize=(6,6))
protocol_key = ['U0_1', 'U0_2', 'gamma_1', 'gamma_2', 'beta_1', 'beta_2', 'd_beta_1', 'd_beta_2', 'phi_1_x', 'phi_2_x', 'phi_1_dcx', 'phi_2_dcx', 'M_12', 'x_c']
contour_plt = plt


phi_1_dcx_index = protocol_key.index('phi_1_dcx')
phi_2_dcx_index = protocol_key.index('phi_2_dcx')
phi_1_dc_i = self.protocol.get_params(t)[phi_1_dcx_index]
phi_2_dc_i = self.protocol.get_params(t)[phi_2_dcx_index]
slice_values = [0, 0, phi_1_dc_i, phi_2_dc_i]

modified_manual_domain = [(manual_domain[0][1], manual_domain[0][0]), \
                              (manual_domain[1][1], manual_domain[1][0])]
U, X_mesh = self.system.lattice(t, resolution, axes=(0, 1),\
                                    manual_domain=modified_manual_domain, slice_values = slice_values)

# 5. find vmin, vmax, x_min, x_max, y_min and y_max
vmin, vmax = np.min(U), np.max(U)
x_min, x_max = np.min(X), np.max(X)
y_min, y_max = np.min(Y), np.max(Y)

all_state = self.sim.output.all_state['states']
phi_1_dc = all_state[..., 2, 0]
phi_2_dc = all_state[..., 3, 0]
all_state = all_state[:, :, (0, 1), :]

N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
X, Y = X_mesh[0], X_mesh[1]
which_axes = [np.s_[..., i, 0] for i in range(N_dim)]
x_array = [all_state[item] for item in which_axes]
x = x_array[0]
y = x_array[1]
state_lookup = separate_by_state(all_state, has_velocity = True)
pColor = {"00": "#3FC7F2", "01": "#FCEF51", "10": "#BEDC8C", "11": "#F7B6D2"}
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F7B6D2"}

cont = ax.contourf(X, Y, U, 40, vmin = 0, vmax = 400, cmap = "afmhot")
scat = [ax.scatter(x[state_lookup[key], 120], y[state_lookup[key], 120], c = pColor[key]) for key in state_lookup]

# phi_1dc, phi_2dc and gamma

In [ ]:
general_analysis_tool = importlib.reload(general_analysis_tool)
general_analysis_tool.show_phi_dc_with_time(simResult['cfqr'], params)

# save result

In [ ]:
print(f"current comment: {params['comment']}")
response = input("Are you happy with the current comment? If not, give me a new comment.")
if response == "y":
    pass
else:
    params['comment'] = response

In [ ]:
params['comment'] = "test for save all_states"

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery", save_all_state = False)

# work_analysis

In [ ]:
selected_work_distribution = simResult["work_distribution"] < -0.2
# selected_work_distribution = simResult["work_distribution"] > 1.5
# selected_work_distribution = np.logical_and(simResult["work_distribution"] > -0.2, simResult["work_distribution"] < 1.5)

In [ ]:
keyStep_work_distribution = empty([simulation.ntrials, protocol_time_index_array])

In [ ]:
keyStep_work_distribution = np.empty([10, 2])

In [ ]:
keyStep_work_distribution

In [ ]:
len(protocol_time_index_array - )

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 1000
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
selected_particles = simResult["cfqr"].sim.output.all_state["states"][selected_work_distribution, ...]
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(selected_particles, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

In [ ]:
# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
step_time_array = np.array(simResult["cfqr"].sim.output.all_state["step_indices"]) * params['dt']
potential_array = []
for _index, _t in enumerate(step_time_array):
    _params_at_t = simResult['cfqr'].system.protocol.get_params(_t)
    def cfq_potential(state):
        # print(state)
        phi_1, phi_2, phi_1dc, phi_2dc = state
        return coupled_flux_qubit_non_linear_approx_pot(phi_1, phi_2, phi_1dc, phi_2dc, _params_at_t)
    vectorized_cfq_potential = np.vectorize(cfq_potential, signature='(4)->()')
    potential_array.append(vectorized_cfq_potential(selected_particles[:, _index, :, 0]))
    
    

In [ ]:
plt.plot(potential_array)
plt.show()

In [ ]:
protocol_time_array, protocol_time_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
protocol_all_time_index_array = np.array(range(0, int(np.array(protocol_time_array[-1]) / params['dt']) + 1))
protocol_all_time_array  = protocol_all_time_index_array * params['dt']

In [ ]:
minimum_value_data_points = []
importlib.reload(minimum_value_of_potential)
for t in protocol_all_time_array[::1000]:
    minimum_value_data = minimum_value_of_potential.find_minimum_for_all_potential_2(cfqr, t,  beta_1, beta_2, d_beta_1, d_beta_2, verbose = False)
    minimum_value_data_points.append(minimum_value_data[1])

In [ ]:
minimum_value_of_potential = importlib.reload(minimum_value_of_potential)

In [ ]:
minimum_value_data_points = np.array(minimum_value_data_points)

In [ ]:
offset_value = minimum_value_data_points[1:] - minimum_value_data_points[:-1]

In [ ]:
all_minimum_value_at_t = []

for t in protocol_time_array:
    minimum_value_data = minimum_value_of_potential.find_all_minimum_points_for_all_potential(cfqr, t,  beta_1, beta_2, d_beta_1, d_beta_2, verbose = False, guess=[[-2, -2], [-2, 2], [2, -2], [2, 2]])
    all_minimum_value_at_t.append([minimum_value_data, t])

In [ ]:
all_minimum_value_at_t

In [ ]:
plt.plot(minimum_value_data_points)

In [ ]:
plt.plot(protocol_all_time_array[::10], minimum_value_data_points)

In [ ]:
protocol_time_array

# one particle work done

In [ ]:
all_states = simResult['cfqr'].sim.output.all_state['states']

In [ ]:
index_of_00 = simResult['cfqr'].getIndexOfParticles()["00"]
index_of_01 = simResult['cfqr'].getIndexOfParticles()["01"]
index_of_10 = simResult['cfqr'].getIndexOfParticles()["10"]
index_of_11 = simResult['cfqr'].getIndexOfParticles()["11"]
all_states_00_i1 = all_states[index_of_00][5, ...]

In [ ]:
_, _, protocol_time_array, protocol_time_index_array = simResult['cfqr'].createProtocolTimeArray(protocol_list, params)
protocol_all_time_index_array = np.array(range(0, int(np.array(protocol_time_array[-1]) / params['dt']) + 1))
protocol_all_time_array  = protocol_all_time_index_array * params['dt']
protocol_all_time_array

In [ ]:
protocal_key_time_array = simResult['cfqr'].protocol_all_time_array[simResult['cfqr'].sim.output.all_state['step_indices']]
KE_of_the_particle = simResult['cfqr'].system.get_kinetic_energy(all_states_00_i1)
PE_of_the_particle = [simResult['cfqr'].system.get_potential(_s, _t ) for _t, _s in  zip(protocol_all_time_array[::200], all_states_00_i1)]
WD_of_the_particle = simResult['cfqr'].sim.work_dist_time_array_whole_process[index_of_00][5]

In [ ]:
all_states_00_i1.shape

In [ ]:
protocol_all_time_array.shape

In [ ]:
len(protocal_key_time_array)

In [ ]:
len(WD_of_the_particle)

In [ ]:
plt.plot(protocol_all_time_array[::100], KE_of_the_particle, label = "KE")
plt.plot(protocol_all_time_array[::200], PE_of_the_particle - min(PE_of_the_particle) , label = "PE")
plt.plot(protocol_all_time_array[::200], WD_of_the_particle[::200], label = "WD")
plt.legend()

In [ ]:
plt.plot(protocal_key_time_array, KE_of_the_particle, label = "KE")
plt.plot(protocal_key_time_array, PE_of_the_particle - min(PE_of_the_particle) , label = "PE")
# plt.plot(protocal_key_time_array, WD_of_the_particle[::200], label = "WD")
plt.ylabel(r"energy $(k_BT)$")
plt.xlabel(r"time")
plt.legend()

In [ ]:
np.mean(simResult['cfqr'].system.get_kinetic_energy(all_states[:,-1,...]))

In [ ]:
def findPotential(cfqr, _t, state):
    _params_at_t = cfqr.system.protocol.get_params(_t)
    def cfq_potential(state):
        phi_1, phi_2, phi_1dc, phi_2dc = state
        return coupled_flux_qubit_non_linear_approx_pot_break_down(phi_1, phi_2, phi_1dc, phi_2dc, _params_at_t)
    return np.array(list(cfq_potential(state).values()))

In [ ]:


for i in range(1, len(step_time_array)):
    # print(step_time_array[i], selected_particles[i, :])
    U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[i-1, :])
    # U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[i-1, :])
    U_b[i-1] = findPotential(simResult["cfqr"],  step_time_array[i-1], selected_particles[i-1, :])

In [ ]:
u1_1, u2_1, u3_1, u4_1, u1_2, u2_2, u3_2, u4_2, u5 = [U_a[:, i] for i in range(0, 9)]


In [ ]:
# plt.plot(u1_1)
# plt.plot(u2_1)
# plt.plot(u3_1)
# plt.plot(u4_1)


In [ ]:
np.sum(U_a - U_b)

In [ ]:
step_time_array[1:].shape

In [ ]:
selected_particles.shape

In [ ]:
np.zeros((max(100, int(2*NT)), self.potential.N_dim, 2))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

phi_1x = 1.2
phi_1xdc = np.pi/2 
def f(phi_1, phi_1dc):
    return (phi_1 - phi_1x)**2 / 7  + (phi_1dc - phi_1xdc)**2 - 2 * np.cos(phi_1) * np.cos(phi_1x)

v_func = np.vectorize(f)    # major key!6

x, y = np.meshgrid(np.linspace(-6, 6, 100),
                   np.linspace(-6, 6, 100))

fig, ax = plt.subplots(1)
ax.contour(y, x, v_func(x, y), 20)
plt.show()